In [65]:
import pandas as pd
import os
from keras.models import Sequential
from keras.layers import Dense,Conv2D,BatchNormalization,MaxPool2D
from keras.layers import Dropout, Flatten
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn import svm
# Moduli di scikit-learn
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, ParameterGrid, train_test_split, StratifiedShuffleSplit, StratifiedKFold, StratifiedGroupKFold
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, ParameterGrid, train_test_split, StratifiedShuffleSplit
import ml_utilities
import ml_visualization
import ml_utilities
import ml_visualization

In [19]:
TESTS_PATH = "./tests/"
INFO_PATH = "./tests/experiments_info"
DATASET_TEST_PATH = 'DBs/PenDigits/pendigits_te.txt'
DATASET_PATH = 'DBs/PenDigits/pendigits_tr.txt'

In [24]:
def read_data():
    feat_count = 16
    data_path = DATASET_PATH # Impostare il percorso corretto
    patterns, labels = ml_utilities.load_labeled_dataset_from_txt(data_path, feat_count)
    test_path = DATASET_TEST_PATH
    x_test = ml_utilities.load_unlabeled_dataset_from_txt(test_path, feature_count)
    return patterns, labels, x_test

In [ ]:
def normalize_data(dataset_patterns, test_x):

    scaler = MinMaxScaler()
    # alldata = np.concatenate((dataset_patterns, test_x))
    # transformed_data = scaler.fit_transform(alldata)

    transformed_data_train = scaler.fit_transform(dataset_patterns)
    transformed_data_test = scaler.fit_transform(test_x)


    # dataset_patterns = transformed_data[:442]
    # test_x = transformed_data[442:]
    return transformed_data_train, transformed_data_test

In [31]:
def feature_extractor(x_grid):
    SIZE = x_grid.shape[1]
    activation = 'sigmoid'
    feature = Sequential()

    feature.add(Conv2D(32, 3, activation = activation, padding = 'same', input_shape = (SIZE, SIZE, 1)))
    feature.add(BatchNormalization())

    feature.add(Conv2D(32, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
    feature.add(BatchNormalization())
    feature.add(MaxPool2D())

    feature.add(Flatten())

    return feature

In [153]:

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1],  #Regularization parameter. Providing only two as SVM is slow
            'kernel': ['rbf','linear']
        }
    },
    'SVC': {
        'model': SVC(gamma='auto'),
        'params' : {
            'C': [1],  #Regularization parameter. Providing only two as SVM is slow
            'kernel': ['rbf','linear']
        }
    },
    'KNeighbors': {
        'model': KNeighborsClassifier(),
        'params' : {
            'n_neighbors': [1],
            'weights': ['uniform'],
            'algorithm': ['auto'],
            'metric': ['minkowski'],

        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1]  #Regularization. . Providing only two as LR can be slow
        }
    }
}
scores=[]

In [87]:
# Caricamento del dataset
feature_count = 16
dataset_path = 'DBs/PenDigits/pendigits_tr.txt'  # Impostare il percorso corretto

dataset_patterns, dataset_labels = ml_utilities.load_labeled_dataset_from_txt(dataset_path, feature_count)
print('Shape dataset:', dataset_patterns.shape)
print('Shape labels:', dataset_labels.shape)

Shape dataset: (442, 16)
Shape labels: (442,)


In [154]:
np.random.seed(42)

x_train, y_train, x_test = read_data()
x_grid, x_not_use, y_grid, y_not_use = train_test_split(x_train,y_train, test_size=0.2,random_state=42)
for data_test_size in [0.2,0.3]:
    for split in [1,2,3,4,5,6,7,8,9,10]:
        cross_val = StratifiedShuffleSplit(n_splits=split, test_size=data_test_size, random_state=42)

        for model_name, mp in model_params.items():
            grid =  GridSearchCV(estimator=mp['model'],
                                 param_grid=mp['params'],
                                 cv=cross_val, n_jobs=16,
                                 return_train_score=False)

            grid.fit(x_train,y_train)
            scores.append({
                'data_test_size': data_test_size,
                'split': split,
                'model': model_name,
                'best_score': grid.best_score_,
                'mean_test_score': grid.cv_results_['mean_test_score'],
                'best_params': grid.best_params_,
            })

        df = pd.DataFrame(scores,columns=['data_test_size','split','model','best_score','mean_test_score','best_params',])

In [155]:
df

,data_test_size,split,model,best_score,mean_test_score,best_params
0,0.2,1,svm,0.696629,"[0.1348314606741573, 0.6966292134831461]","{'C': 1, 'kernel': 'linear'}"
1,0.2,1,SVC,0.696629,"[0.1348314606741573, 0.6966292134831461]","{'C': 1, 'kernel': 'linear'}"
2,0.2,1,KNeighbors,0.764045,[0.7640449438202247],"{'algorithm': 'auto', 'metric': 'minkowski', '..."
3,0.2,1,logistic_regression,0.707865,[0.7078651685393258],{'C': 1}
4,0.2,2,svm,0.735955,"[0.1348314606741573, 0.7359550561797753]","{'C': 1, 'kernel': 'linear'}"
...,...,...,...,...,...,...
75,0.3,9,logistic_regression,0.734336,[0.7343358395989975],{'C': 1}
76,0.3,10,svm,0.729323,"[0.3398496240601504, 0.7293233082706767]","{'C': 1, 'kernel': 'linear'}"
77,0.3,10,SVC,0.729323,"[0.3398496240601504, 0.7293233082706767]","{'C': 1, 'kernel': 'linear'}"
78,0.3,10,KNeighbors,0.780451,[0.780451127819549],"{'algorithm': 'auto', 'metric': 'minkowski', '..."
